# Notebook 7: Upload Corrected Assemblies<a class="tocSkip">

**Upload the final reassembled fastas as a submission**
    
    
**The steps that we will take are:**
1. Import Statements & Global Variable Definitions
2. Read In Data Tables
3. Select Correct Assemblies To Upload
4. Upload QC Data
5. Extract QC Metrics 

# Import Statements & Global Variable Definitions

In [46]:
%%capture
%pip install --upgrade --no-cache-dir git+https://github.com/DataBiosphere/ssds

In [2]:
import terra_notebook_utils as tnu
import terra_pandas as tp
import pandas as pd
import os
from ssds import deployment
from pprint import pprint
import gcsfs
import seaborn as sns
%matplotlib inline

## Function Definitions

In [3]:
def clean_quast_string(input_string):
    input_string = str(input_string)
    rstrip_str   = input_string.rstrip()
    rtn_str      = rstrip_str.split(' ')[-1]
    
    return rtn_str

## Global Variable Definitions

In [4]:
## submission_id is generated by uuid (gives unique ID to each submission)
submission_id       = "4CE43AFC-BA7F-480F-B073-8A0C303E9DAB"

## submission_name lends some recognizability to the submission
submission_name     = "YEAR_1_ASSEMBLIES_V3"

ds = deployment._GSStaging(google_billing_project="firecloud-cgl")   # _GSStaging()
destintation_bucket = "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf" # _GSStaging()

## Where the submission files will actually land (we may add one directory level later)
dest_full_path      = os.path.join(destintation_bucket, "submissions", submission_id + "--" + submission_name)

## Set Environment Variables

In [5]:
# Get the Google billing project name and workspace name
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_Reassembly
Workspace storage bucket: gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/


# Read In Data Tables

## Read In Original QC Samples

In [6]:
qc_sample_df = tp.table_to_dataframe("qc_sample")

qc_sample_df.head()

,asmgeneMaternalSummary,quastPaternalSummary,mat_ilmn,mat_final_fasta,child_ilmn,is_male,allResults,asmgenePaternalSummary,quastMaternalSummary,merquryQV,pat_ilmn,sample_name,dipcallVCF,dipcallBED,pat_final_fasta,yakSummary
qc_sample_id,,,,,,,,,,,,,,,,
HG002_downsampled,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,True,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG002,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...
HG002_full_v0.14,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,True,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG002,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...
HG00438,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,False,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG00438,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...
HG005,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,True,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG005,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...
HG00621,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,True,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG00621,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...


## Read In Corrected/Split QC Samples

In [7]:
qc_split_sample_df = tp.table_to_dataframe("qc_split_sample")

qc_split_sample_df.head()

,asmgeneMaternalSummary,quastPaternalSummary,mat_ilmn,mat_final_fasta,is_male,allResults,asmgenePaternalSummary,quastMaternalSummary,pat_ilmn,sample_name,dipcallVCF,dipcallBED,pat_final_fasta,yakSummary,child_ilmn,merquryQV
qc_split_sample_id,,,,,,,,,,,,,,,,
HG01123,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,FALSE,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG01123,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/s...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,NaN,NaN
HG01358,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,TRUE,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG01358,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/s...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/d...


# Select Correct Assemblies To Upload

In [8]:
## Check how many rows we have
len(qc_sample_df.index)

48

In [9]:
qc_sample_df.drop(["HG01123", "HG01358", "HG002_downsampled"], inplace=True)

In [30]:
## Check how many rows we have now that we dropped three rows
len(qc_sample_df.index)

45

**Join the dataframes to get final samples for upload**

In [10]:
upload_df = pd.concat([qc_sample_df, qc_split_sample_df], sort=False)

In [11]:
## Check number of rows one last time (should be 47 now)
len(upload_df.index)

47

## Upload Directly From GCP w/ SSDS

In [15]:
## Loop through rows of data table, upload fastas
for index, row in upload_df.iterrows():
    
    sample_id   = row.name
    sample_name = row['sample_name']
    

    mat_fa_fp = row['mat_final_fasta']
    pat_fa_fp = row['pat_final_fasta']

    mat_fa_fn = os.path.basename(mat_fa_fp)
    pat_fa_fn = os.path.basename(pat_fa_fp)
        
    mat_subm_path = f"{sample_name}/assemblies/year1_freeze_assembly_v3/{mat_fa_fn}"
    pat_subm_path = f"{sample_name}/assemblies/year1_freeze_assembly_v3/{pat_fa_fn}"
                
    ds.copy(mat_fa_fp, submission_id, submission_name, mat_subm_path)
    ds.copy(pat_fa_fp, submission_id, submission_name, pat_subm_path)

2021-03-19 02:24:28::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/592e8a80-c574-49d2-8a14-4a9fba2232c1/finalizeAssembly/fcf717b6-ac97-434c-98bc-4ec6b69fcfb9/call-renameContigsAddMT/HG002.maternal.f1_assembly_v2.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4CE43AFC-BA7F-480F-B073-8A0C303E9DAB--YEAR_1_ASSEMBLIES_V3/HG002/assemblies/year1_freeze_assembly_v3/HG002.maternal.f1_assembly_v2.fa.gz
2021-03-19 02:24:43::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c738faa4-a3b7-45b0-9ee5-3578085d38e5/finalizeAssembly/5f2c050c-0be2-413a-80b3-8bfbae35d612/call-renameContigsAddMT/HG002.paternal.f1_assembly_v2.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4CE43AFC-BA7F-480F-B073-8A0C303E9DAB--YEAR_1_ASSEMBLIES_V3/HG002/assemblies/year1_freeze_assembly_v3/HG002.paternal.f1_assembly_v2.fa.gz
2021-03-19 02:24:55::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/592e8a80-c574-49d2-8a14-4a9fba2232c1/finalizeAssembly/05b3ca1c-9f57

2021-03-19 02:28:05::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c738faa4-a3b7-45b0-9ee5-3578085d38e5/finalizeAssembly/56c332ea-b121-4632-a64d-581705e3c4aa/call-renameContigsAddMT/HG01106.paternal.f1_assembly_v2.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4CE43AFC-BA7F-480F-B073-8A0C303E9DAB--YEAR_1_ASSEMBLIES_V3/HG01106/assemblies/year1_freeze_assembly_v3/HG01106.paternal.f1_assembly_v2.fa.gz
2021-03-19 02:28:17::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/592e8a80-c574-49d2-8a14-4a9fba2232c1/finalizeAssembly/82e3de5c-4f8e-48b2-8ad6-89ebae0e810e/call-renameContigsAddMT/HG01109.maternal.f1_assembly_v2.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4CE43AFC-BA7F-480F-B073-8A0C303E9DAB--YEAR_1_ASSEMBLIES_V3/HG01109/assemblies/year1_freeze_assembly_v3/HG01109.maternal.f1_assembly_v2.fa.gz
2021-03-19 02:28:29::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c738faa4-a3b7-45b0-9ee5-3578085d38e5/finalizeAssembly/9

2021-03-19 02:31:37::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/592e8a80-c574-49d2-8a14-4a9fba2232c1/finalizeAssembly/382c2a6c-4a1c-4149-b553-6e697c3c56f0/call-renameContigsAddMT/HG02055.maternal.f1_assembly_v2.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4CE43AFC-BA7F-480F-B073-8A0C303E9DAB--YEAR_1_ASSEMBLIES_V3/HG02055/assemblies/year1_freeze_assembly_v3/HG02055.maternal.f1_assembly_v2.fa.gz
2021-03-19 02:31:48::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c738faa4-a3b7-45b0-9ee5-3578085d38e5/finalizeAssembly/3f464f2a-e3e3-4631-a381-cd9a6afdfd32/call-renameContigsAddMT/HG02055.paternal.f1_assembly_v2.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4CE43AFC-BA7F-480F-B073-8A0C303E9DAB--YEAR_1_ASSEMBLIES_V3/HG02055/assemblies/year1_freeze_assembly_v3/HG02055.paternal.f1_assembly_v2.fa.gz
2021-03-19 02:31:59::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/592e8a80-c574-49d2-8a14-4a9fba2232c1/finalizeAssembly/f

2021-03-19 02:35:05::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c738faa4-a3b7-45b0-9ee5-3578085d38e5/finalizeAssembly/62b16b4f-2e61-4199-93b5-72d79d4ed359/call-renameContigsAddMT/HG02622.paternal.f1_assembly_v2.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4CE43AFC-BA7F-480F-B073-8A0C303E9DAB--YEAR_1_ASSEMBLIES_V3/HG02622/assemblies/year1_freeze_assembly_v3/HG02622.paternal.f1_assembly_v2.fa.gz
2021-03-19 02:35:16::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/592e8a80-c574-49d2-8a14-4a9fba2232c1/finalizeAssembly/155fdcd5-6cbc-4e6c-8874-14211f0aa42f/call-renameContigsAddMT/HG02630.maternal.f1_assembly_v2.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4CE43AFC-BA7F-480F-B073-8A0C303E9DAB--YEAR_1_ASSEMBLIES_V3/HG02630/assemblies/year1_freeze_assembly_v3/HG02630.maternal.f1_assembly_v2.fa.gz
2021-03-19 02:35:27::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c738faa4-a3b7-45b0-9ee5-3578085d38e5/finalizeAssembly/2

2021-03-19 02:38:37::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/592e8a80-c574-49d2-8a14-4a9fba2232c1/finalizeAssembly/188d98e7-5117-4f66-8f20-e40f09928cdb/call-renameContigsAddMT/HG03516.maternal.f1_assembly_v2.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4CE43AFC-BA7F-480F-B073-8A0C303E9DAB--YEAR_1_ASSEMBLIES_V3/HG03516/assemblies/year1_freeze_assembly_v3/HG03516.maternal.f1_assembly_v2.fa.gz
2021-03-19 02:38:47::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c738faa4-a3b7-45b0-9ee5-3578085d38e5/finalizeAssembly/95a535fe-b209-428e-8582-51f95468d169/call-renameContigsAddMT/HG03516.paternal.f1_assembly_v2.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4CE43AFC-BA7F-480F-B073-8A0C303E9DAB--YEAR_1_ASSEMBLIES_V3/HG03516/assemblies/year1_freeze_assembly_v3/HG03516.paternal.f1_assembly_v2.fa.gz
2021-03-19 02:38:58::INFO  Copied gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/592e8a80-c574-49d2-8a14-4a9fba2232c1/finalizeAssembly/f

# Upload QC Data
## Download Then Extract QC Tarballs

In [13]:
! mkdir qc_data
%cd qc_data

mkdir: cannot create directory ‘qc_data’: File exists
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data


In [14]:
qc_upload_df = upload_df.copy()
qc_upload_df.drop(["HG005"], inplace=True)

In [18]:
for index, row in qc_upload_df.iterrows():
    
    sample_name = row['sample_name']
    
    ## Copy down QC results
    results_tar_fp = row['allResults']
    ! gsutil cp {results_tar_fp} .

    ## Unzip and remove original tar.gz file
    qc_tar_fn = os.path.basename(results_tar_fp)
    ! tar xzvf {qc_tar_fn}
    ! rm {qc_tar_fn}
    
    #filename, file_extension = os.path.splitext(qc_tar_fn)
    filename     = os.path.basename(qc_tar_fn)
    index_of_dot = filename.index('.')
    folder_name  = filename[:index_of_dot]

    ## Move to a more standard name
    ! mv {folder_name} {sample_name}/assembly_qc

Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a7695be6-0d2e-4b2d-9a45-66169cb24c9e/standardQualityControl/9103c0d4-ab33-48e6-b2b5-f0b23ab346c6/call-consolidate/glob-86aa2126671e0b3e99d2a48a23fb36d9/HG002_StandardQC.tar.gz...
/ [1 files][ 14.6 GiB/ 14.6 GiB]   45.0 MiB/s                                   
Operation completed over 1 objects/14.6 GiB.                                     
HG002_StandardQC/
HG002_StandardQC/asmgene/
HG002_StandardQC/asmgene/pat/
HG002_StandardQC/asmgene/pat/HG002.paternal.f1_assembly_v2.gene_stats.txt
HG002_StandardQC/asmgene/mat/
HG002_StandardQC/asmgene/mat/HG002.maternal.f1_assembly_v2.gene_stats.txt
HG002_StandardQC/yak/
HG002_StandardQC/yak/HG002.f1_assembly_v2.mat.yak.switch-error.txt
HG002_StandardQC/yak/HG002.f1_assembly_v2.pat.yak.switch-error.txt
HG002_StandardQC/yak/HG002.f1_assembly_v2.pat.yak.qv.txt
HG002_StandardQC/yak/HG002.f1_assembly_v2.summary.txt
HG002_StandardQC/yak/HG002.f1_assembly_v2.mat.yak.qv.txt
HG002_StandardQC/dipcall_v0.1

HG002_StandardQC/merqury/HG002.f1_assembly_v2.merqury.asm.paternal.hapmers.spectra-cn.ln.png
HG002_StandardQC/merqury/HG002.f1_assembly_v2.merqury.asm.spectra-cn.fl.png
HG002_StandardQC/merqury/inherited_hapmers.fl.png
HG002_StandardQC/merqury/HG002.f1_assembly_v2.merqury.completeness.stats
HG002_StandardQC/merqury/inherited_hapmers.st.png
HG002_StandardQC/merqury/HG002.f1_assembly_v2.merqury.asm.maternal.hapmers.spectra-cn.fl.png
HG002_StandardQC/merqury/HG002.f1_assembly_v2.merqury.altHap.100_20000.switches.txt
HG002_StandardQC/merqury/HG002.f1_assembly_v2.merqury.asm.maternal.hapmers.tdf
HG002_StandardQC/merqury/HG002.f1_assembly_v2.merqury.altHap.spectra-cn.hist
HG002_StandardQC/merqury/HG002.f1_assembly_v2.merqury.asm.contig.sizes
HG002_StandardQC/merqury/HG002.f1_assembly_v2.merqury.altHap.paternal.hapmers.spectra-hap-cn.hist
HG002_StandardQC/merqury/HG002.f1_assembly_v2.merqury.altHap.100_20000.phased_block.bed
HG002_StandardQC/merqury/HG002.f1_assembly_v2.merqury.spectra-cn.fl.

HG00438_StandardQC/merqury/HG00438.f1_assembly_v2.merqury.asm.100_20000.switch.bed
HG00438_StandardQC/merqury/HG00438.f1_assembly_v2.merqury.altHap.maternal.hapmers.tdf
HG00438_StandardQC/merqury/HG00438.f1_assembly_v2.merqury.asm.maternal.hapmers.spectra-cn.st.png
HG00438_StandardQC/merqury/HG00438.f1_assembly_v2.merqury.hapmers.blob.png
HG00438_StandardQC/merqury/HG00438.f1_assembly_v2.merqury.asm.maternal.hapmers.spectra-hap-cn.hist
HG00438_StandardQC/merqury/HG00438.f1_assembly_v2.merqury.altHap.maternal.hapmers.spectra-cn.ln.png
HG00438_StandardQC/merqury/HG00438.f1_assembly_v2.merqury.altHap.paternal.hapmers.tdf
HG00438_StandardQC/merqury/HG00438.f1_assembly_v2.merqury.asm.100_20000.phased_block.bed
HG00438_StandardQC/merqury/HG00438.f1_assembly_v2.merqury.asm.100_20000.phased_block.sizes
HG00438_StandardQC/merqury/HG00438.f1_assembly_v2.merqury.asm.100_20000.switches.txt
HG00438_StandardQC/merqury/HG00438.f1_assembly_v2.merqury.spectra-asm.ln.png
HG00438_StandardQC/merqury/HG004

HG00621_StandardQC/merqury/HG00621.f1_assembly_v2.merqury.altHap.maternal.hapmers.spectra-cn.st.png
HG00621_StandardQC/merqury/HG00621.f1_assembly_v2.merqury.asm.100_20000.phased_block.counts
HG00621_StandardQC/merqury/HG00621.f1_assembly_v2.merqury.altHap.block.N.png
HG00621_StandardQC/merqury/HG00621.f1_assembly_v2.merqury.completeness.stats
HG00621_StandardQC/merqury/HG00621.f1_assembly_v2.merqury.altHap.100_20000.switches.txt
HG00621_StandardQC/merqury/HG00621.f1_assembly_v2.merqury.altHap.maternal.hapmers.bed
HG00621_StandardQC/merqury/HG00621.f1_assembly_v2.merqury.altHap.100_20000.phased_block.blob.png
HG00621_StandardQC/merqury/HG00621.f1_assembly_v2.merqury.asm.spectra-cn.fl.png
HG00621_StandardQC/merqury/HG00621.f1_assembly_v2.merqury.altHap.continuity.N.png
HG00621_StandardQC/merqury/HG00621.f1_assembly_v2.merqury.altHap.spectra-cn.hist
HG00621_StandardQC/merqury/maternal.inherited.hist
HG00621_StandardQC/merqury/HG00621.f1_assembly_v2.merqury.altHap.paternal.hapmers.spectra

HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.hap1.bam
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.hap2.var.gz
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.hap2.bed
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.hap1.bed
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.hap2.var.gz.vst
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.hap2.bam
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.hap1.sam.gz.log
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.hap2.sam.gz
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.hap2.paf.gz.log
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.hap1.paf.gz
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.dip.vcf.gz
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.hap1.var.gz.vst
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.hap1.paf.gz.log
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.tmp.bed
HG00621_StandardQC/dipcall_v0.2/HG00621.f1_assembly_v2.dip.

...

\ [1 files][ 13.0 GiB/ 13.0 GiB]   42.1 MiB/s                                   
Operation completed over 1 objects/13.0 GiB.                                     
NA20129_StandardQC/
NA20129_StandardQC/asmgene/
NA20129_StandardQC/asmgene/mat/
NA20129_StandardQC/asmgene/mat/NA20129.maternal.f1_assembly_v2.gene_stats.txt
NA20129_StandardQC/asmgene/pat/
NA20129_StandardQC/asmgene/pat/NA20129.paternal.f1_assembly_v2.gene_stats.txt
NA20129_StandardQC/dipcall_v0.2/
NA20129_StandardQC/dipcall_v0.2/NA20129.f1_assembly_v2.hap2.var.gz
NA20129_StandardQC/dipcall_v0.2/NA20129.f1_assembly_v2.hap2.sam.gz.log
NA20129_StandardQC/dipcall_v0.2/NA20129.f1_assembly_v2.hap1.paf.gz.log
NA20129_StandardQC/dipcall_v0.2/NA20129.f1_assembly_v2.hap2.bam
NA20129_StandardQC/dipcall_v0.2/NA20129.f1_assembly_v2.hap1.sam.gz.log
NA20129_StandardQC/dipcall_v0.2/NA20129.f1_assembly_v2.hap2.var.gz.vst
NA20129_StandardQC/dipcall_v0.2/NA20129.f1_assembly_v2.hap1.var.gz.vst
NA20129_StandardQC/dipcall_v0.2/NA20129.f1_assembl

NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.hap2.sam.gz.log
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.hap1.paf.gz.log
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.hap2.bam
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.hap1.sam.gz.log
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.hap2.var.gz.vst
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.hap1.var.gz.vst
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.dip.bed
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.dip.vcf.gz
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.hap1.paf.gz
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.hap1.var.gz
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.hap2.paf.gz.log
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.hap1.bam
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.hap2.sam.gz
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assembly_v2.hap1.sam.gz
NA20129_StandardQC/dipcall_v0.1/NA20129.f1_assemb

NA21309_StandardQC/merqury/NA21309.f1_assembly_v2.merqury.altHap.100_20000.phased_block.sizes
NA21309_StandardQC/merqury/NA21309.f1_assembly_v2.merqury.asm.contig.sizes
NA21309_StandardQC/merqury/NA21309.f1_assembly_v2.merqury.altHap.100_20000.phased_block.blob.png
NA21309_StandardQC/merqury/NA21309.f1_assembly_v2.merqury.altHap.maternal.hapmers.tdf
NA21309_StandardQC/merqury/NA21309.f1_assembly_v2.merqury.altHap.maternal.hapmers.spectra-cn.st.png
NA21309_StandardQC/merqury/NA21309.f1_assembly_v2.merqury.asm.maternal.hapmers.spectra-cn.ln.png
NA21309_StandardQC/merqury/NA21309.f1_assembly_v2.merqury.asm.maternal.hapmers.spectra-cn.fl.png
NA21309_StandardQC/merqury/NA21309.f1_assembly_v2.merqury.asm.paternal.hapmers.spectra-cn.fl.png
NA21309_StandardQC/merqury/NA21309.f1_assembly_v2.merqury.altHap.paternal.hapmers.spectra-cn.st.png
NA21309_StandardQC/merqury/NA21309.f1_assembly_v2.merqury.altHap.paternal.hapmers.tdf
NA21309_StandardQC/merqury/paternal.inherited.hist
NA21309_StandardQC/m

HG01123_StandardQC/quast/pat/report.pdf
HG01123_StandardQC/quast/pat/transposed_report.tex
HG01123_StandardQC/quast/pat/report.tex
HG01123_StandardQC/quast/pat/report.tsv
HG01123_StandardQC/quast/pat/quast.log
HG01123_StandardQC/quast/pat/transposed_report.tsv
HG01123_StandardQC/quast/pat/report.txt
HG01123_StandardQC/quast/pat/basic_stats/
HG01123_StandardQC/quast/pat/basic_stats/GC_content_plot.pdf
HG01123_StandardQC/quast/pat/basic_stats/HG01123.paternal.f1_assembly_v2_GC_content_plot.pdf
HG01123_StandardQC/quast/pat/basic_stats/Nx_plot.pdf
HG01123_StandardQC/quast/pat/basic_stats/cumulative_plot.pdf
HG01123_StandardQC/quast/pat/basic_stats/NGx_plot.pdf
HG01123_StandardQC/quast/mat/
HG01123_StandardQC/quast/mat/transposed_report.txt
HG01123_StandardQC/quast/mat/report.html
HG01123_StandardQC/quast/mat/report.pdf
HG01123_StandardQC/quast/mat/transposed_report.tex
HG01123_StandardQC/quast/mat/report.tex
HG01123_StandardQC/quast/mat/report.tsv
HG01123_StandardQC/quast/mat/quast.log
HG0

HG01358_StandardQC/merqury/paternal_not_maternal.hist
HG01358_StandardQC/merqury/HG01358.f1_assembly_v2.merqury.altHap.maternal.hapmers.spectra-cn.st.png
HG01358_StandardQC/merqury/HG01358.f1_assembly_v2.merqury.altHap.spectra-cn.st.png
HG01358_StandardQC/merqury/HG01358.f1_assembly_v2.merqury.altHap.paternal.hapmers.spectra-cn.st.png
HG01358_StandardQC/merqury/HG01358.f1_assembly_v2.merqury.asm.continuity.N.png
HG01358_StandardQC/merqury/HG01358.f1_assembly_v2.merqury.asm.paternal.hapmers.spectra-cn.st.png
HG01358_StandardQC/merqury/HG01358.f1_assembly_v2.merqury.altHap.qv
HG01358_StandardQC/merqury/HG01358.f1_assembly_v2.merqury.asm.paternal.hapmers.spectra-hap-cn.hist
HG01358_StandardQC/merqury/HG01358.f1_assembly_v2.merqury.altHap.paternal.hapmers.tdf
HG01358_StandardQC/merqury/inherited_hapmers.ln.png
HG01358_StandardQC/merqury/HG01358.f1_assembly_v2.merqury.completeness.stats
HG01358_StandardQC/merqury/HG01358.f1_assembly_v2.merqury.asm.100_20000.phased_block.sizes
HG01358_Standa

In [52]:
hg005_df = upload_df.copy()
hg005_df

,asmgeneMaternalSummary,quastPaternalSummary,mat_ilmn,mat_final_fasta,child_ilmn,is_male,allResults,asmgenePaternalSummary,quastMaternalSummary,merquryQV,pat_ilmn,sample_name,dipcallVCF,dipcallBED,pat_final_fasta,yakSummary
HG002_full_v0.14,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,True,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG002,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...
HG00438,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,False,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG00438,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...
HG005,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,True,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG005,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...
HG00621,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,True,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG00621,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...
HG00673,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,True,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG00673,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...
HG00733,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0

In [54]:
for index, row in hg005_df.iterrows():
    
    sample_name = row['sample_name']
    
    if sample_name == "HG005":
        
        sample_name = row['sample_name']

        ## Copy down QC results
        results_tar_fp = row['allResults']
        ! gsutil cp {results_tar_fp} .

        ## Unzip and remove original tar.gz file
        qc_tar_fn = os.path.basename(results_tar_fp)
        ! tar xzvf {qc_tar_fn}
        ! rm {qc_tar_fn}

        #filename, file_extension = os.path.splitext(qc_tar_fn)
        filename     = os.path.basename(qc_tar_fn)
        index_of_dot = filename.index('.')
        folder_name  = filename[:index_of_dot]

        ## Move to a more standard name
        ! mv {folder_name} {sample_name}/assembly_qc

Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5999c1a9-993d-4c50-a70e-36d7c00be0cf/standardQualityControl/d515fef0-6b94-4471-9f8e-1ed816d4cc46/call-consolidate/glob-86aa2126671e0b3e99d2a48a23fb36d9/HG005_StandardQC.tar.gz...
- [1 files][ 13.0 GiB/ 13.0 GiB]   43.6 MiB/s                                   
Operation completed over 1 objects/13.0 GiB.                                     
HG005_StandardQC/
HG005_StandardQC/dipcall_v0.2/
HG005_StandardQC/dipcall_v0.2/HG005.f1_assembly_v2.hap1.paf.gz
HG005_StandardQC/dipcall_v0.2/HG005.f1_assembly_v2.hap1.bam
HG005_StandardQC/dipcall_v0.2/HG005.f1_assembly_v2.hap1.var.gz
HG005_StandardQC/dipcall_v0.2/HG005.f1_assembly_v2.hap1.sam.gz
HG005_StandardQC/dipcall_v0.2/HG005.f1_assembly_v2.tmp.bed
HG005_StandardQC/dipcall_v0.2/HG005.f1_assembly_v2.hap1.var.gz.vst
HG005_StandardQC/dipcall_v0.2/HG005.f1_assembly_v2.hap2.paf.gz
HG005_StandardQC/dipcall_v0.2/HG005.f1_assembly_v2.hap1.paf.gz.log
HG005_StandardQC/dipcall_v0.2/HG005.f1_assembly_v2.

HG005_StandardQC/merqury/inherited_hapmers.st.png
HG005_StandardQC/merqury/HG005.f1_assembly_v2.merqury.asm.paternal.hapmers.spectra-cn.ln.png
HG005_StandardQC/merqury/HG005.f1_assembly_v2.merqury.altHap.paternal.hapmers.spectra-cn.fl.png
HG005_StandardQC/merqury/HG005.f1_assembly_v2.merqury.altHap.100_20000.switch.bed
HG005_StandardQC/merqury/paternal_not_maternal.hist
HG005_StandardQC/merqury/HG005.f1_assembly_v2.merqury.spectra-cn.st.png
HG005_StandardQC/merqury/HG005.f1_assembly_v2.merqury.altHap.spectra-cn.st.png
HG005_StandardQC/merqury/HG005.f1_assembly_v2.merqury.spectra-asm.fl.png
HG005_StandardQC/merqury/HG005.f1_assembly_v2.merqury.asm.paternal.hapmers.tdf
HG005_StandardQC/merqury/HG005.f1_assembly_v2.merqury.altHap.100_20000.phased_block.blob.png
HG005_StandardQC/merqury/HG005.f1_assembly_v2.merqury.asm.spectra-cn.st.png
HG005_StandardQC/merqury/HG005.f1_assembly_v2.merqury.asm.qv
HG005_StandardQC/merqury/HG005.f1_assembly_v2.merqury.asm.continuity.N.png
HG005_StandardQC/me

In [56]:
! ls | wc -l

47


## Upload As Submission

In [57]:
%cd ..

/home/jupyter-user/notebooks/HPRC_Reassembly/edit


In [59]:
! ssds staging upload \
    --deployment gcp \
    --submission-id de582218-9b22-4169-b595-7034d5fff6c8 \
    --name V2_ASSEMBLY_QC \
    qc_data

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/quast/pat/report.tsv to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG01258_StandardQC/quast/pat/report.tsv
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/quast/pat/report.txt to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG01258_StandardQC/quast/pat/report.txt
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/quast/pat/quast.log to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG01258_StandardQC/quast/pat/quast.log
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/quast/pat/report.pdf to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/sub

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/merqury/inherited_hapmers.st.png to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG01258_StandardQC/merqury/inherited_hapmers.st.png
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.altHap.contig.sizes to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.altHap.contig.sizes
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.altHap.paternal.hapmers.spectra-hap-cn.hist to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.altHap.paternal.hapmers.spectra-cn.fl.png to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.altHap.paternal.hapmers.spectra-cn.fl.png
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.asm.block.N.png to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.asm.block.N.png
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.altHap.qv to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.asm.100_20000.switch.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.asm.100_20000.switch.bed
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/merqury/maternal_not_paternal.hist to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG01258_StandardQC/merqury/maternal_not_paternal.hist
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.altHap.maternal.hapmers.spectra-cn.fl.png to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG01258_StandardQC/merqury/HG01258.f1_

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.asm.spectra-cn.ln.png to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.asm.spectra-cn.ln.png
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.altHap.maternal.hapmers.spectra-hap-cn.hist to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.altHap.maternal.hapmers.spectra-hap-cn.hist
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG01258_StandardQC/merqury/HG01258.f1_assembly_v2.merqury.altHap.maternal.hapmers.tdf to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4

...

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/dipcall_v0.2/HG02717.f1_assembly_v2.pair.vcf.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717_StandardQC/dipcall_v0.2/HG02717.f1_assembly_v2.pair.vcf.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/dipcall_v0.2/HG02717.f1_assembly_v2.hap2.bam to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717_StandardQC/dipcall_v0.2/HG02717.f1_assembly_v2.hap2.bam
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/dipcall_v0.2/HG02717.f1_assembly_v2.hap2.paf.gz.log to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717_StandardQC/dipcall_v0.2/HG02717.f1_assembly_v2.hap2.paf.gz.log
INFO:ssd

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.qv to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.qv
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.asm.100_20000.switch.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.asm.100_20000.switch.bed
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.asm.100_20000.phased_block.sizes to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717_StandardQC/merqury/HG0271

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.asm.scaff.sizes to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.asm.scaff.sizes
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.altHap.spectra-cn.fl.png to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.altHap.spectra-cn.fl.png
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.altHap.paternal.hapmers.spectra-hap-cn.hist to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.altHap.paternal.hapmers.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.altHap.paternal.hapmers.bed
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.altHap.qv to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.altHap.qv
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/merqury/HG02717.f1_assembly_v2.merqury.asm.paternal.hapmers.spectra-cn.ln.png to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/dipcall_v0.1/HG02717.f1_assembly_v2.hap1.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717_StandardQC/dipcall_v0.1/HG02717.f1_assembly_v2.hap1.bed
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/dipcall_v0.1/HG02717.f1_assembly_v2.hap2.var.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717_StandardQC/dipcall_v0.1/HG02717.f1_assembly_v2.hap2.var.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/qc_data/HG02717_StandardQC/dipcall_v0.1/HG02717.f1_assembly_v2.hap1.sam.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/de582218-9b22-4169-b595-7034d5fff6c8--V2_ASSEMBLY_QC/HG02717_StandardQC/dipcall_v0.1/HG02717.f1_assembly_v2.hap1.sam.gz
INFO:ssds.storag

# Extract QC Metrics

## Add QUAST Info

In [13]:
qc_extract_df = upload_df.copy()
qc_extract_df

,asmgeneMaternalSummary,quastPaternalSummary,mat_ilmn,mat_final_fasta,child_ilmn,is_male,allResults,asmgenePaternalSummary,quastMaternalSummary,merquryQV,pat_ilmn,sample_name,dipcallVCF,dipcallBED,pat_final_fasta,yakSummary
HG002_full_v0.14,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,True,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG002,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...
HG00438,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,False,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG00438,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...
HG005,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,True,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG005,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...
HG00621,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,True,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG00621,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...
HG00673,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,True,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG00673,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...
HG00733,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0

In [14]:
metric_df = qc_extract_df[['sample_name', 'dipcallBED', 'dipcallVCF']].copy()


metric_df["mat_num_contigs"] = ""
metric_df["mat_total_len"]   = ""
metric_df["mat_N50"]         = ""
metric_df["mat_L50"]         = ""

metric_df["pat_num_contigs"] = ""
metric_df["pat_total_len"]   = ""
metric_df["pat_N50"]         = ""
metric_df["pat_L50"]         = ""

In [15]:
for index, row in qc_extract_df.iterrows():
    
    mat_quast_fp = row['quastMaternalSummary']
    mat_quast_df = pd.read_csv(mat_quast_fp, sep='\t')

    metric_df['mat_num_contigs'][index] = clean_quast_string(mat_quast_df.iloc[13, ].values[0])
    metric_df['mat_total_len'][index]   = clean_quast_string(mat_quast_df.iloc[7, ].values[0])
    metric_df['mat_N50'][index]         = clean_quast_string(mat_quast_df.iloc[18, ].values[0])
    metric_df['mat_L50'][index]         = clean_quast_string(mat_quast_df.iloc[22, ].values[0])   
    
    ## Paternal
    pat_quast_fp = row['quastPaternalSummary']
    pat_quast_df = pd.read_csv(pat_quast_fp, sep='\t')
    
    metric_df['pat_num_contigs'][index] = clean_quast_string(pat_quast_df.iloc[13, ].values[0])
    metric_df['pat_total_len'][index]   = clean_quast_string(pat_quast_df.iloc[7, ].values[0])
    metric_df['pat_N50'][index]         = clean_quast_string(pat_quast_df.iloc[18, ].values[0])
    metric_df['pat_L50'][index]         = clean_quast_string(pat_quast_df.iloc[22, ].values[0])

## Add asmgene info

In [16]:
metric_df["mat_full_sgl"]     = ""
metric_df["mat_full_dup"]     = ""
metric_df["mat_frag"]         = ""

metric_df["pat_full_sgl"]     = ""
metric_df["pat_full_dup"]     = ""
metric_df["pat_frag"]         = ""

In [17]:
for index, row in qc_extract_df.iterrows():
    ## Maternal
    mat_asm_fp = row['asmgeneMaternalSummary']
    mat_asm_df = pd.read_csv(mat_asm_fp, sep='\t')

    is_full_sgl_val = mat_asm_df['Metric'] == 'full_sgl'
    is_full_dup_val = mat_asm_df['Metric'] == 'full_dup'
    is_frag_val     = mat_asm_df['Metric'] == 'frag'

    metric_df["mat_full_sgl"][index] = mat_asm_df[is_full_sgl_val].iloc[:,3].values[0]
    metric_df["mat_full_dup"][index] = mat_asm_df[is_full_dup_val].iloc[:,3].values[0]
    metric_df["mat_frag"][index]     = mat_asm_df[is_frag_val].iloc[:,3].values[0]

    
    ## Paternal
    pat_asm_fp = row['asmgenePaternalSummary']
    pat_asm_df = pd.read_csv(pat_asm_fp, sep='\t')

    is_full_sgl_val = pat_asm_df['Metric'] == 'full_sgl'
    is_full_dup_val = pat_asm_df['Metric'] == 'full_dup'
    is_frag_val     = pat_asm_df['Metric'] == 'frag'

    metric_df["pat_full_sgl"][index] = pat_asm_df[is_full_sgl_val].iloc[:,3].values[0]
    metric_df["pat_full_dup"][index] = pat_asm_df[is_full_dup_val].iloc[:,3].values[0]
    metric_df["pat_frag"][index]     = pat_asm_df[is_frag_val].iloc[:,3].values[0]

## Add YAK Info

In [18]:
metric_df["mat_switch_err_rate"]  = ""
metric_df["mat_hamming_err_rate"] = ""
metric_df["mat_qv"]               = ""

metric_df["pat_switch_err_rate"]  = ""
metric_df["pat_hamming_err_rate"] = ""
metric_df["pat_qv"]               = ""

In [19]:
for index, row in qc_extract_df.iterrows():
    ## Maternal
    yak_fp = row['yakSummary']
    yak_df = pd.read_csv(yak_fp, sep=',', header=None)
    
    ## Where we run QV calculations (have child Illumina data)
    if yak_df.iloc[0,0] == '# mat qv':
        metric_df["mat_switch_err_rate"][index]  = yak_df.iloc[11,:].values[0].split('\t')[3]
        metric_df["mat_hamming_err_rate"][index] = yak_df.iloc[12,:].values[0].split('\t')[3]
        metric_df["mat_qv"][index]               = yak_df.iloc[4,:].values[0].split('\t')[2]

        metric_df["pat_switch_err_rate"][index]  = yak_df.iloc[15,:].values[0].split('\t')[3]
        metric_df["pat_hamming_err_rate"][index] = yak_df.iloc[16,:].values[0].split('\t')[3]
        metric_df["pat_qv"][index]               = yak_df.iloc[9,:].values[0].split('\t')[2]
    
    ## Where we don't run QV calculations (do not have child Illumina data)
    else:
        metric_df["mat_switch_err_rate"][index]  = yak_df.iloc[1,:].values[0].split('\t')[3]
        metric_df["mat_hamming_err_rate"][index] = yak_df.iloc[2,:].values[0].split('\t')[3]
        metric_df["mat_qv"][index]               = "Not_Calc"

        metric_df["pat_switch_err_rate"][index]  = yak_df.iloc[5,:].values[0].split('\t')[3]
        metric_df["pat_hamming_err_rate"][index] = yak_df.iloc[6,:].values[0].split('\t')[3]
        metric_df["pat_qv"][index]               = "Not_Calc"

In [20]:
metric_df

,sample_name,dipcallBED,dipcallVCF,mat_num_contigs,mat_total_len,mat_N50,mat_L50,pat_num_contigs,pat_total_len,pat_N50,...,mat_frag,pat_full_sgl,pat_full_dup,pat_frag,mat_switch_err_rate,mat_hamming_err_rate,mat_qv,pat_switch_err_rate,pat_hamming_err_rate,pat_qv
HG002_full_v0.14,HG002,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/a...,458,3141612006,86523007,13,648,2939821391,61505630,...,4,34673,164,6,0.009134,0.008612,53.012,0.008554,0.008604,53.749
HG00438,HG00438,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,259,3035929804,54936949,19,278,3025358313,48061544,...,10,34727,131,9,0.009714,0.010295,52.836,0.009502,0.008803,52.845
HG005,HG005,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,541,3031523002,69736411,17,811,2942569812,58303677,...,9,34667,129,9,0.006806,0.005658,53.090,0.002831,0.002081,52.982
HG00621,HG00621,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,254,3023109422,50294217,19,295,2906296479,54673245,...,8,34679,142,8,0.007039,0.005985,52.838,0.004673,0.005548,53.015
HG00673,HG00673,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,353,3053794682,29077043,35,436,2926307565,34843587,...,11,34593,189,14,0.009603,0.009333,53.151,0.003116,0.002701,53.352
HG00733,HG00733,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/0...,617,3026801692,44647564,24,755,3043880851,40994157,...,11,34722,149,11,0.009791,0.009964,54.823,0.009760,0.009330,54.558
HG00735,HG00735,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/7...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/7...,251,3038195502,56474489,18,321,3034561332,53422923,...,7,34671,145,11,0.006650,0.007120,52.720,0.009743,0.008950,52.711
HG00741,HG00741,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/7...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/7...,307,3037308484,41001116,23,311,3031070223,51040418,...,16,34706,130,12,0.005852,0.005426,52.511,0.010897,0.012607,52.363
HG01071,HG01071,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/7...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/7...,238,3012820263,50125412,17,367,3059213820,55592407,...,14,34606,162,17,0.009084,0.011651,51.285,0.003776,0.004007,51.147
HG01106,HG01106,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/7...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/7...,283,3036387987,47714433,20,310,2928126274,57173280,...,7,34735,136,8,0.004466,0.009752,53.394,0.003519,0.003244,53.751


## Copy Metrics As CSV To Bucket

In [21]:
output_metric_fn = "2021_03_20_Y1_assemblies_QC.csv"
metric_df.to_csv(output_metric_fn, index=False)

In [22]:
! gsutil cp {output_metric_fn} {bucket}{output_metric_fn}

Copying file://2021_03_20_Y1_assemblies_QC.csv [Content-Type=text/csv]...
/ [1 files][ 28.5 KiB/ 28.5 KiB]                                                
Operation completed over 1 objects/28.5 KiB.                                     


## Plot a few metrics